In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.colors as colors # Coloring
import seaborn as sns

In [2]:
metadata = pd.read_csv('pbmc_preds.csv',index_col=0)
metadata = metadata.loc[metadata['virus']=='HIV']
BNab = metadata.loc[metadata['Condition']=='BNab'].index
Cont = metadata.loc[metadata['Condition']=='control'].index

FileNotFoundError: [Errno 2] No such file or directory: 'pbmc_preds.csv'

In [ ]:
NK = pd.read_csv('HIV_NK_sigm.txt',sep='\t',index_col=0)
Monocytes = pd.read_csv('HIV_Monocytes_sigm.txt',sep='\t',index_col=0)
CD4T = pd.read_csv('HIV_CD4Tcells_sigm.txt',sep='\t',index_col=0)
CD8T = pd.read_csv('HIV_CD8Tcells_sigm.txt',sep='\t',index_col=0)
Bcell = pd.read_csv('HIV_Bcells_sigm.txt',sep='\t',index_col=0)
Den = pd.read_csv('HIV_Dendritics_sigm.txt',sep='\t',index_col=0)
HIV = pd.read_csv('HIV_pbmc_tpm.csv',sep=',',index_col=0)
samplename = HIV.columns
genename = HIV.index

In [ ]:
HIV = HIV.values
HIV = np.log2(HIV+1)
# mms = MinMaxScaler()
# HIV = mms.fit_transform(HIV)
HIV = pd.DataFrame(HIV,columns=samplename,index=genename).T

In [ ]:
def fdr(p_vals):

    from scipy.stats import rankdata
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr
def volcano_data(data, groupA, groupB):
    A = data[groupA]
    B = data[groupB]
    genes = data.index
    volcano = np.zeros((len(genes),2))

    for i,gene in enumerate(genes):
        A_gene = np.array(A.loc[gene])
        B_gene = np.array(B.loc[gene])
        volcano[i,0] = (stats.ttest_ind(A_gene,B_gene)[1])
        volcano[i,1] = np.log(A_gene.mean()/B_gene.mean())
    volcano[:,0] = -np.log10(fdr(volcano[:,0]))
    volcano = pd.DataFrame(volcano, columns=['-Log10 q-value','Log Fold change'], index=genes)
    return volcano
def volcano_plot(data, fc_threshold, p_threshold):
    data.columns = ['y','x']
    data['group'] = 'black'
    data.loc[(data.x > fc_threshold)&(data.y > p_threshold),'group'] = 'tab:red'
    data.loc[(data.x < -fc_threshold)&(data.y > p_threshold),'group'] = 'tab:blue'
    data.loc[data.y < p_threshold,'group'] = 'dimgrey' #阈值以下点为灰色
    xmin = -8
    xmax = 8
    ymin = 0
    ymax = 8
    #绘制散点图
    sns.set()
    fig = plt.figure(figsize=(6,7),dpi=300) 
    ax = fig.add_subplot()
    ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax), title='')
    ax.scatter(data['x'], data['y'], s=2, c=data['group'])
    ax.scatter(data.loc['RAB11FIP5']['x'], data.loc['RAB11FIP5']['y'], c='green', marker='D')
    ax.spines['right'].set_visible(False) #去掉右边框
    ax.spines['top'].set_visible(False) #去掉上边框

    #水平和竖直线
    ax.vlines(-fc_threshold, ymin, ymax, color='dimgrey',linestyle='dashed', linewidth=1) #画竖直线
    ax.vlines(fc_threshold, ymin, ymax, color='dimgrey',linestyle='dashed', linewidth=1) #画竖直线
    ax.hlines(p_threshold, xmin, xmax, color='dimgrey',linestyle='dashed', linewidth=1) #画竖水平线

    ax.set_xticks(range(xmin,xmax,2)) #设置x轴刻度起点和步长
    ax.set_yticks(range(ymin,ymax,1)) #设置y轴刻度起点和步长
    plt.xlabel('Log Fold Change')
    plt.ylabel('-Log10 q-value')
    plt.show()

In [ ]:
HIV_volcano = volcano_data(HIV, BNab, Cont)
volcano_plot(HIV_volcano, 1, 2)
print(HIV_volcano.loc['RAB11FIP5'])


NK_volcano = volcano_data(NK, BNab, Cont)
volcano_plot(NK_volcano, 1, 2)
print(NK_volcano.loc['RAB11FIP5'])


Bcell_volcano = volcano_data(Bcell, BNab, Cont)
volcano_plot(Bcell_volcano, 1, 2)
print(Bcell_volcano.loc['RAB11FIP5'])


Den_volcano = volcano_data(Den, BNab, Cont)
# volcano_plot(Den_volcano, 1, 2)
print(Den_volcano.loc['RAB11FIP5'])


CD4T_volcano = volcano_data(CD4T, BNab, Cont)
# volcano_plot(CD4T_volcano, 1, 2)
print(CD4T_volcano.loc['RAB11FIP5'])


CD8T_volcano = volcano_data(CD8T, BNab, Cont)
# volcano_plot(CD8T_volcano, 1, 2)
print(CD8T_volcano.loc['RAB11FIP5'])


Mon_volcano = volcano_data(Monocytes, BNab, Cont)
# volcano_plot(Mon_volcano, 1, 2)
print(Mon_volcano.loc['RAB11FIP5'])
#volcano_plot(HIV_volcano,1,2)

In [ ]:
# volcano_plot(NK_volcano,1,2)

In [ ]:
df = pd.concat((Bcell.loc['RAB11FIP5'],CD4T.loc['RAB11FIP5'],
                CD8T.loc['RAB11FIP5'],Den.loc['RAB11FIP5'],
                Monocytes.loc['RAB11FIP5'],NK.loc['RAB11FIP5']))
df = pd.DataFrame(df.values, columns=['Expression Value'],index=df.index)

In [ ]:
Type = []
for x in df.index:
    if x in BNab:
        Type.append('BNab')
    elif x in Cont:
        Type.append('control')
Cell = ['Bcell' for i in range(92)]+ \
       ['CD4Tcell' for i in range(92)]+ \
       ['CD8Tcell' for i in range(92)]+ \
       ['Dendritics' for i in range(92)]+ \
       ['Monocyte' for i in range(92)]+ \
       ['NK' for i in range(92)]

In [ ]:
df['Type'] = Type
df['Cell Type'] = Cell

In [ ]:
plt.figure(dpi=300)
ax = sns.boxplot(x="Cell Type", y="Expression Value", hue="Type",hue_order=['control','BNab'],data=df, palette="Set3", dodge=True)
#plt.ylim(0.,1)
plt.xticks(rotation=70)
plt.xlabel('')
plt.show()